<a href="https://colab.research.google.com/github/Dharshaan24/vechical-detection-/blob/main/theos_ocr_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Set these variables with your own values
Drop the video to the colab instance filesystem, rename it to the **INPUT_VIDEO** filename.

In [1]:
ALGORITHM = 'yolov7' # set the algorithm, such as 'yolov7'
ALGORITHM_VERSION = 'tiny' # set the algorithm version, such as 'tiny'
WEIGHTS = 'license-plates:experiment-1:best' # set the weights tag, such as 'license-plates:experiment-1:best'

OCR_MODEL_SIZE = 'large' # set the OCR model size, possible values are 'small', 'medium' and 'large'
OCR_MODEL_TYPE = 'printed' # set the OCR model type if using OCR_MODEL_SIZE = 'large', possible values are 'str', 'printed' and 'handwritten'
OCR_MODEL_ACCURACY = 'best' # set the OCR model accuracy if using OCR_MODEL_SIZE = 'large', possible values are 'base', 'medium' and 'best'
OCR_CLASSES = ['license-plate'] # set the class names that the OCR model should read from, such as ['license-plate']

INPUT_VIDEO = 'video.mp4'
OUTPUT_VIDEO = 'output.mp4'

## 2. Run the setup code

In [2]:
from getpass import getpass
import subprocess

PYTHON_VERSION = '3.10'

def run(command):
  p  = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
  while True:
      output = p.stdout.readline()
      if output == '' and p.poll() is not None:
          break
      if output:
          print(output.strip())

def python(script):
  p = subprocess.Popen(f'python{PYTHON_VERSION} -c "{script}"', shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
  while True:
      output = p.stdout.readline()
      if output == '' and p.poll() is not None:
          break
      if output:
          print(output.strip())

def login():
  username = input('[theos] username: ')
  password = getpass('[theos] password: ')
  run(f'theos login -u {username} -p {password}')

def set_project():
  key = getpass('[theos] project key: ')
  run(f'theos set-project {key}')

def install_algorithm(algorithm, version, weights):
  run(f'theos install {algorithm} --version {version} --weights {weights}')

INSTALL = f'sudo apt install tesseract-ocr && wget https://bootstrap.pypa.io/get-pip.py && python{PYTHON_VERSION} get-pip.py && python{PYTHON_VERSION} -m pip install --upgrade pip && python{PYTHON_VERSION} -m pip install theos-ai[ocr]==0.0.36'
SETUP = 'theos setup --version v1 --subfield object-detection'
SCRIPT = f'''
from theos.computer_vision.object_detection.utils import draw
from theos.computer_vision import ocr
from theos.client import Client
from tqdm import tqdm
import torch
import json
import time
import cv2
import os

if torch.cuda.is_available():
  torch.cuda.empty_cache()

ALGORITHM = '{ALGORITHM}'
ALGORITHM_VERSION = '{ALGORITHM_VERSION}'
WEIGHTS = '{WEIGHTS}'

OCR_MODEL_SIZE = '{OCR_MODEL_SIZE}'
OCR_MODEL_TYPE = '{OCR_MODEL_TYPE}'
OCR_MODEL_ACCURACY = '{OCR_MODEL_ACCURACY}'
OCR_CLASSES = {OCR_CLASSES}

INPUT_VIDEO = '{INPUT_VIDEO}'
OUTPUT_VIDEO = '{OUTPUT_VIDEO}'
client = Client(inputs='.', outputs='.')
yolov7 = client.get(ALGORITHM, version=ALGORITHM_VERSION)
yolov7.load_weights(WEIGHTS)
yolov7.to_gpu()

if OCR_MODEL_TYPE and OCR_MODEL_ACCURACY:
  ocr_model = ocr.load(size=OCR_MODEL_SIZE, model_type=OCR_MODEL_TYPE, accuracy=OCR_MODEL_ACCURACY)
else:
  ocr_model = ocr.load(size=OCR_MODEL_SIZE)

if OCR_MODEL_SIZE == 'large':
  ocr_model.to_gpu()

video = cv2.VideoCapture(INPUT_VIDEO)
width  = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video.get(cv2.CAP_PROP_FPS))
frames_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
output = cv2.VideoWriter(OUTPUT_VIDEO, fourcc, fps, (width, height))

if video.isOpened() == False:
    print('[!] error opening the video')

print('[theos] detecting video...')
print()
pbar = tqdm(total=frames_count, unit=' frames', dynamic_ncols=True, position=0, leave=True)

while video.isOpened():
    ret, frame = video.read()
    if ret == True:
        detections = yolov7.detect(frame)
        detections = ocr_model.read(frame, detections=detections, classes=OCR_CLASSES)
        detected_frame = draw(frame, detections, alpha=0.15)
        output.write(detected_frame)
        pbar.update(1)
    else:
        break

pbar.close()
print()
video.release()
output.release()
yolov7.unload()
client.close()

print('[theos] you can now download the output video.')
'''

## 3. Install the required packages

In [3]:
run(INSTALL)
run(SETUP)



Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: 

## 4. Log in to your account

In [4]:
login()

[theos] username: dharshaan
[theos] password: ··········

Theos CLI


[theos] login succeeded


# 5. Set your theos project
Enter your Theos **project key**.


In [8]:
!set_project(08a57c60-8b00-4f0d-80b4-f)

/bin/bash: -c: line 1: syntax error near unexpected token `08a57c60-8b00-4f0d-80b4-f'
/bin/bash: -c: line 1: `set_project(08a57c60-8b00-4f0d-80b4-f)'


# 6. Install the algorithm and weights

In [10]:
!theos install yolov7 --version tiny --weights coco:default:best

[theos] downloading yolov7 code:

100% 272k/272k [00:00<00:00, 351kB/s]

[theos] extracting yolov7 code...
[theos] downloading coco weights for the tiny version of yolov7:

100% 12.1M/12.1M [00:01<00:00, 6.58MB/s]

[theos] coco default:best weights downloaded successfully ✓


# 7. Detect and read

This script will detect and perform OCR on your video and save the output video to your machine.

In [1]:
ALGORITHM = 'yolov7' # set the algorithm, such as 'yolov7'
ALGORITHM_VERSION = 'tiny' # set the algorithm version, such as 'tiny'
WEIGHTS = 'coco:default:best' # set the weights tag, such as 'license-plates:experiment-1:best'

OCR_MODEL_SIZE = 'large' # set the OCR model size, possible values are 'small', 'medium' and 'large'
OCR_MODEL_TYPE = 'printed' # set the OCR model type if using OCR_MODEL_SIZE = 'large', possible values are 'str', 'printed' and 'handwritten'
OCR_MODEL_ACCURACY = 'best' # set the OCR model accuracy if using OCR_MODEL_SIZE = 'large', possible values are 'base', 'medium' and 'best'
OCR_CLASSES = ['license plate'] # set the class names that the OCR model should read from, such as ['license-plate']

INPUT_VIDEO = 'video.mp4'
OUTPUT_VIDEO = 'output.mp4'

# 8. Download the output video

In [ ]:
from google.colab import files
print('[theos] downloading output video...\n')
files.download(OUTPUT_VIDEO)